## Creating the MongoDB Database

In [1]:
#!pip install pymongo

In [2]:
import pymongo

Start the mongo daemon!

In [3]:
from pymongo import MongoClient
import pandas as pd

In [4]:
import numpy as np

In [5]:
client = MongoClient('localhost', 27017)
db = client.project2 #Create the database

In [6]:
#Create the Group Member collection
collection = db.groupMember

In [7]:
courses = pd.read_csv('courses.csv', encoding = "ISO-8859-1", engine='python') 
fam_members = pd.read_csv('familyMembers.csv', encoding = "ISO-8859-1", engine='python') 
group_members = pd.read_csv('groupMembers.csv', encoding = "ISO-8859-1", engine='python') 
pets = pd.read_csv('pets.csv', encoding = "ISO-8859-1", engine='python') 

In [8]:
group_members['age'] = group_members['age'].apply(lambda x: int(str(x)))
group_members['petCount'] = group_members['petCount'].apply(lambda x: int(str(x)))
group_members = group_members.astype({'inState': 'bool'})

In [9]:
courses['course number'] = courses['course number'].apply(lambda x: int(str(x)))
courses['class capacity'] = courses['class capacity'].apply(lambda x: int(str(x)))

In [10]:
fam_members['age'] = fam_members['age'].apply(lambda x: int(str(x)))
fam_members = fam_members.astype({'enrolledAtUVA': 'bool'})

In [11]:
pets['age'] = pets['age'].apply(lambda x: int(float(str(x))))

In [12]:
def correct_encoding(dictionary):
    """Correct the encoding of python dictionaries so they can be encoded to mongodb
    inputs
    -------
    dictionary : dictionary instance to add as document
    output
    -------
    new : new dictionary with (hopefully) corrected encodings"""

    new = {}
    for key1, val1 in dictionary.items():
        # Nested dictionaries
        if isinstance(val1, dict):
            val1 = correct_encoding(val1)

        if isinstance(val1, np.bool_):
            val1 = bool(val1)

        if isinstance(val1, np.int64):
            val1 = int(val1)

        if isinstance(val1, np.float64):
            val1 = float(val1)

        new[key1] = val1

    return new

### Restructuring data into python dicts

In [13]:
for i in range(len(group_members)):    # loop through group members 
    curMem = group_members.iloc[i]
    #print(curMem)
    cur_dict = {}
    for x in range(len(curMem)): 
       # print(x, curMem.index[x])
        if x!=0 and x < 7:              # name - siblingCount props
            cur_dict[curMem.index[x]] = curMem[x]
        if x == 10 or x >19:           # petCount and major - inState props
            cur_dict[curMem.index[x]] = curMem[x]
        if x == 11 or x == 12:           # pets 
            if not pd.isna(curMem[x]):   # if not null find in the pets df 
                petId = curMem[x]
                curPet = pd.DataFrame(pets.loc[pets['id '] == petId]).reset_index()
                if not cur_dict.get('pets'): 
                    cur_dict['pets'] = {}
               # print(curPet['name'])
                cur_dict['pets'][curMem.index[x]] = {}
                cur_dict['pets'][curMem.index[x]]['name'] = curPet['name'][0]
                cur_dict['pets'][curMem.index[x]]['species'] = curPet['species'][0]
                cur_dict['pets'][curMem.index[x]]['breed'] = curPet['breed'][0]
                cur_dict['pets'][curMem.index[x]]['gender'] = curPet['gender'][0]
                cur_dict['pets'][curMem.index[x]]['age'] = curPet['age'][0]
                cur_dict['pets'][curMem.index[x]]['color'] = curPet['color'][0]
        if x > 12 and x <20:           # courses 
            if not pd.isna(curMem[x]): # if not null, find in courses df 
                courseId = curMem[x]
                curCourse = pd.DataFrame(courses.loc[courses['id'] == courseId])
                if not cur_dict.get('courses'): 
                    cur_dict['courses'] = {}
                #print(curCourse['course mnemonic'][courseId])
                cur_dict['courses'][curMem.index[x]] = {}
                cur_dict['courses'][curMem.index[x]]['mnemonic'] = curCourse['course mnemonic'][courseId]
                cur_dict['courses'][curMem.index[x]]['courseNumber'] = int(curCourse['course number'][courseId])
                cur_dict['courses'][curMem.index[x]]['department'] = curCourse['department'][courseId]
                cur_dict['courses'][curMem.index[x]]['classTitle'] = curCourse['class title'][courseId]
                cur_dict['courses'][curMem.index[x]]['professor'] = curCourse['professor'][courseId]
                cur_dict['courses'][curMem.index[x]]['meetingDay(s)'] = curCourse['meeting day(s)'][courseId]
                cur_dict['courses'][curMem.index[x]]['type'] = curCourse['type'][courseId]
                cur_dict['courses'][curMem.index[x]]['classCapacity'] = int(curCourse['class capacity'][courseId])
                cur_dict['courses'][curMem.index[x]]['courseDescription'] = curCourse['course description'][courseId]
                #print(cur_dict.get('courses'))
    for y in range(len(fam_members)): 
        curFam = fam_members.iloc[y]
        if curFam['groupMemberRelative'] == curMem['id']: 
            if not cur_dict.get('fam_members'): 
                cur_dict['fam_members'] = {}
            cur_dict['fam_members'][curFam['name']] = {}
            cur_dict['fam_members'][curFam['name']]['relation'] = curFam['relation']
            cur_dict['fam_members'][curFam['name']]['age'] = int(curFam['age'])
            cur_dict['fam_members'][curFam['name']]['gender'] = curFam['gender']
            cur_dict['fam_members'][curFam['name']]['birthday'] = curFam['birthday']
            cur_dict['fam_members'][curFam['name']]['favIceCream'] = curFam['favIceCream']
            cur_dict['fam_members'][curFam['name']]['petCount'] = curFam['petCount']
            if not pd.isna(curFam['pet1Name']): 
                cur_dict['fam_members'][curFam['name']]['pet1Name'] = pets.loc[pets['id '] == curFam['pet1Name']].to_dict('records')[0]
            if not pd.isna(curFam['pet2Name']): 
                cur_dict['fam_members'][curFam['name']]['pet2Name'] = pets.loc[pets['id '] == curFam['pet2Name']].reset_index().to_dict('records')[0]
            cur_dict['fam_members'][curFam['name']]['enrolledAtUVA'] = bool((curFam['enrolledAtUVA']))
            
    cur_dict = correct_encoding(cur_dict)
    collection.insert_one(cur_dict)

In [14]:
pets.columns

Index(['id ', 'name', 'species', 'breed', 'gender', 'age', 'ownerGroup',
       'color', 'since'],
      dtype='object')

## MongoDB Queries
https://www.geeksforgeeks.org/nested-queries-in-pymongo/

https://pymongo.readthedocs.io/en/stable/examples/aggregation.html

https://www.tutorialspoint.com/mongodb/mongodb_query_document.htm

In [15]:
from bson.son import SON

**How many documents are there?**

In [16]:
collection.count_documents({})

5

**Which group members are taking classes in the CS department?**

In [17]:
for val in collection.find({ "$or": [
         {'courses.course1.department': 'Computer Science'}, 
         {'courses.course2.department':'Computer Science'},
        {'courses.course3.department':'Computer Science'}, 
        {'courses.course4.department':'Computer Science'}, 
         {'courses.course5.department':'Computer Science'},
         {'courses.course6.department':'Computer Science'}, 
        {'courses.course7.department':'Computer Science'}
      ]}): 
    print(val.get('name'))

Anna Grace Calhoun
Daniel Tohti
David Siamon


**What is the average age of the group members’ pets?**

In [18]:
def get_pet_avg(): 
    total = 0 
    count = 0 
    for val in collection.find({'pets.pet1.age': {"$exists":True}}): 
        total += (val.get('pets').get('pet1').get('age'))
        count += 1      
    for val in collection.find({'pets.pet2.age': {"$exists":True}}): 
        total += (val.get('pets').get('pet2').get('age'))
        count += 1
    return round(total/count, 2)
print('The average pet age is', get_pet_avg(), 'years.')

The average pet age is 6.6 years.


In [19]:
for val in collection.aggregate([{"$group": {"_id":None, "AverageValue": {"$avg":"pets.pet1.age"} } }]): 
    print(val)

{'_id': None, 'AverageValue': None}


**Which group members are over 20?**

In [20]:
for val in collection.find({"age": {"$gt": 20}}): 
    print(val.get('name'))
    print(val.get('courses').get('course1').keys())

Dare Nguyen
dict_keys(['mnemonic', 'courseNumber', 'department', 'classTitle', 'professor', 'meetingDay(s)', 'type', 'classCapacity', 'courseDescription'])
Danny Devito
dict_keys(['mnemonic', 'courseNumber', 'department', 'classTitle', 'professor', 'meetingDay(s)', 'type', 'classCapacity', 'courseDescription'])


**What are the names and favorite ice creams of people taking CS 2150?**

In [21]:
for val in collection.find({ "$or": [
         {'courses.course1.classTitle': 'Program & Data Representation'}, 
         {'courses.course2.classTitle':'Program & Data Representation'},
        {'courses.course3.classTitle':'Program & Data Representation'}, 
        {'courses.course4.classTitle':'Program & Data Representation'}, 
         {'courses.course5.classTitle':'Program & Data Representation'},
         {'courses.course6.classTitle':'Program & Data Representation'}, 
        {'courses.course7.classTitle':'Program & Data Representation'}
      ]}): 
    print('Name:', val.get('name'), "\nFavorite Ice Cream:", val.get('favIceCream'))

Name: Daniel Tohti 
Favorite Ice Cream: Mint Chocolate Chip


**Whose family has the most unique ice cream flavors? What are they?**

In [22]:
diff_flavors = {}
for val in collection.find({"name": {"$exists": True}}): 
    diff_flavors[val.get('name')] = []
    for x in val.get('fam_members').values():
        if x.get('favIceCream') not in diff_flavors.get(val.get('name')): 
            diff_flavors[val.get('name')].append(x.get('favIceCream'))
            
length_dict = {key: len(value) for key, value in diff_flavors.items()}
print(length_dict, "\n", diff_flavors.get("Anna Grace Calhoun"))

{'Anna Grace Calhoun': 5, 'Daniel Tohti': 3, 'Dare Nguyen': 4, 'David Siamon': 3, 'Danny Devito': 3} 
 ['Rocky Road', 'Mint Chocolate Chip', 'Cookies and Cream', 'Strawberry', 'Banana']


**What is the average age of dog owners? Cat owners?**

In [23]:
def avg_cd_own_age(): 
    totalDog = 0 
    countDog = 0 
    totalCat = 0 
    countCat = 0 
    for val in collection.find({'pets.pet1': {"$exists":True}}): 
        if val.get('pets').get('pet1').get('species') == 'Dog': 
            totalDog += val.get('age')
            countDog += 1
        if val.get('pets').get('pet1').get('species') == 'Cat': 
            totalCat += val.get('age')
            countCat += 1
    for val in collection.find({'pets.pet2': {"$exists":True}}): 
        if val.get('pets').get('pet2').get('species') == 'Dog': 
            totalDog += val.get('age')
            countDog += 1
        if val.get('pets').get('pet2').get('species') == 'Cat': 
            totalCat += val.get('age')
            countCat += 1
            
    for val in collection.find({"name": {"$exists": True}}): 
        for x in val.get('fam_members').values():
            if x.get('pet1Name'): 
                if x.get('pet1Name').get('species') == 'Dog': 
                    totalDog += x.get('age')
                    countDog += 1
                if x.get('pet1Name').get('species') == 'Cat': 
                    totalCat += x.get('age')
                    countCat += 1
            if x.get('pet2Name'): 
                if x.get('pet2Name').get('species') == 'Dog': 
                    totalDog += x.get('age')
                    countDog += 1
                if x.get('pet2Name').get('species') == 'Cat': 
                    totalCat += x.get('age')
                    countCat += 1
    return totalDog/countDog, totalCat/countCat
        
avg_cd_own_age()

(29.9375, 33.75)

**Which day of the week is most popular for lectures?**

In [24]:
def most_pop_lect_day(): 
    all_days = []
    for val in collection.find({"name": {"$exists": True}}): 
        for x in val.get('courses').values(): 
            if x.get('type') == 'Lecture': 
                all_days.extend(x.get('meetingDay(s)').split())
    all_days = [x for xs in all_days for x in xs.split(',')]
    all_days = [x for x in all_days if x != '']
    return [(i,all_days.count(i)) for i in set(all_days)]

from operator import itemgetter

lis = most_pop_lect_day()
print(lis, '\nMax number of courses on a day:', max(lis,key=itemgetter(1))[1])

[('W', 9), ('Tu', 10), ('Th', 10), ('T', 1), ('F', 6), ('M', 10)] 
Max number of courses on a day: 10
